In [5]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
from datetime import datetime
from urllib.error import HTTPError
import requests

In [6]:
df = pd.read_csv("links.csv", index_col=[0])
df = df.drop_duplicates( keep='last')
df = df.reset_index(drop=True)

In [7]:
df['location'] = df['link'].str.split('/').str[1]
df['type'] = df['link'].str.split('/').str[3]

In [8]:
hdr = {'User-Agent': 'Mozilla/5.0'}
price_df = []
size_df = []
rooms_df = []
num = 1
for i in df['link']:
    site = "https://ingatlan.com" + str(i)
    
    try:
        response = requests.get(site, headers=hdr)
        response.raise_for_status()
    except HTTPError as hp:
        print(hp)
     
    else:
        site= "https://ingatlan.com" + str(i)
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        price = soup.find_all('span', {'fw-bold fs-5 text-nowrap'})
        price = price[0].get_text()
        price = price.strip()
        price_df.append(price)

        rooms = soup.find_all('span', {'fw-bold fs-5'})
        rooms = rooms[1].get_text()
        rooms = rooms.strip()
        rooms_df.append(rooms)

        size = soup.find_all('span', {'fw-bold fs-5'})
        size = size[0].get_text()
        size = size.strip()
        size_df.append(size)
        
        print('done: ' + str(num) + " from " + str(len(df)))
        num = num + 1
        #time.sleep(2.5) #dont want to overwhelm the site

done: 1 from 125
done: 2 from 125
done: 3 from 125
done: 4 from 125
done: 5 from 125
done: 6 from 125
done: 7 from 125
done: 8 from 125
done: 9 from 125
done: 10 from 125
done: 11 from 125
done: 12 from 125
done: 13 from 125
done: 14 from 125
done: 15 from 125
done: 16 from 125
done: 17 from 125
done: 18 from 125
done: 19 from 125
done: 20 from 125
done: 21 from 125
done: 22 from 125
done: 23 from 125
done: 24 from 125
done: 25 from 125
done: 26 from 125
done: 27 from 125
done: 28 from 125
done: 29 from 125
done: 30 from 125
done: 31 from 125
done: 32 from 125
done: 33 from 125
done: 34 from 125
done: 35 from 125
done: 36 from 125
done: 37 from 125
done: 38 from 125
done: 39 from 125
done: 40 from 125
done: 41 from 125
done: 42 from 125
done: 43 from 125
done: 44 from 125
done: 45 from 125
done: 46 from 125
done: 47 from 125
done: 48 from 125
done: 49 from 125
done: 50 from 125
done: 51 from 125
done: 52 from 125
done: 53 from 125
done: 54 from 125
done: 55 from 125
done: 56 from 125
d

In [9]:
df['size'] = size_df
df['rooms'] = rooms_df
df['price'] = price_df

In [10]:

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
df.to_csv("dataframe-" + date + ".csv", sep = ";")